In [ ]:
# import and config
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from zipfile import ZipFile
from pathlib import Path
from os import getcwd, listdir, path
import os 
import scipy.io
import warnings
from typing import Dict, Generic
from sklearn import preprocessing
from typing import TypeVar

warnings.filterwarnings('ignore')

%cd "/home/matej/2-fer/uuzop/projekt"
UNZIP_DATA = False

PATH_CWD = Path(getcwd()) 

PATH_DATA = Path(PATH_CWD, "data")
PATH_DATA_MAT = Path(PATH_DATA, "mat")
PATH_DATA_CNT = Path(PATH_DATA, "cnt")

PATH_ZIP_CNT = Path(PATH_DATA, "5202739.zip")
PATH_ZIP_MAT = Path(PATH_DATA, "5202751.zip")



In [ ]:
# Abstractions

T = TypeVar('T')      # Declare type variable


In [ ]:
# CNT - unzip and restructure data
if  UNZIP_DATA:
    with ZipFile(PATH_ZIP_CNT, 'r') as zip_ref:
        zip_ref.extractall(PATH_DATA_CNT)
    
    zips = [file for file in PATH_DATA_CNT.iterdir() if str(file).endswith(".zip")]
    
    for zip_item in zips:
        
        if not str(zip_item).endswith(".zip"):
            continue
        
        zip_ref = ZipFile(zip_item) # create zipfile object
        
        for cnt_file in zip_ref.namelist()[1:]: # ignore "9/" directory
        
            prefix_number = zip_item.stem # 9
            state_name = Path(cnt_file).stem.lower().split(' ')[0] # "Normal State" -> "normal"
            filename = Path(prefix_number + '_' + state_name + ".cnt")
            
            with open(Path(PATH_DATA_CNT, filename), "wb") as new_file:
                new_file.write(zip_ref.read(cnt_file))
                
    # Delete zips as they were temporary
    for zip_item in zips:
    	os.remove(zip_item)

In [ ]:
# MAT - unzip data
if  UNZIP_DATA:
    with ZipFile(PATH_ZIP_MAT, 'r') as zip_ref:
        zip_ref.extractall(PATH_DATA_MAT)

In [ ]:
mat = scipy.io.loadmat(Path(PATH_DATA_MAT, '1.mat'))
keys = [key for key in mat.keys() if not key.startswith('__')]

[print(key, "with shape",mat[key].shape) for key in keys]

is_fatigue_mask = pd.Series(mat["Class_label"].squeeze() == 1)
keys_entropy = ["FE", "SE", "AE", "PE"]


entropies: Dict[str,pd.DataFrame]={}
for key in keys_entropy:
    entropies[key] = pd.DataFrame(mat[key])
    print(key, "\n",entropies[key].describe()) 
      


In [ ]:
        
def dict_apply_procedture(old_dict: Dict[str, T], procedure) -> Dict[str, T]:
    return {k: procedure(v) for k, v in old_dict.items()}
    
def min_max_dataframe(df: pd.DataFrame):
    return pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df))

for name, entropy in entropies.items():
    plt.scatter(entropy.index.tolist(), entropy.loc[:,1],  label=name)
    plt.legend()

plt.show()

entropies_scaled = dict_apply_procedture(entropies, min_max_dataframe)

for name, entropy in entropies_scaled.items():
    plt.scatter(entropy.index.tolist(), entropy.loc[:,1],  label=name)
    plt.legend()
    
